<a href="https://colab.research.google.com/github/SamikshaSolanke/Netflix-Movie-Recommendation-System/blob/main/notebooks/ntbk3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

In [5]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [6]:
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [8]:
data = pd.concat([df1, df2, df3, df4], ignore_index = True)

In [9]:
data

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3]
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4]
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5]
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6]
...,...,...,...,...,...,...
245,D E C E M B E R,21,Second Act,STX Entertainment,"Peter Segal (director); Justin Zackham, Elaine...",[238]
246,D E C E M B E R,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions /...,Etan Cohen (director/screenplay); Will Ferrell...,[142]
247,D E C E M B E R,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[117]
248,D E C E M B E R,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[207]


In [10]:
data['Cast and crew']

,Cast and crew
0,Adam Robitel (director); Leigh Whannell (scree...
1,Christopher Radcliff (director/screenplay); La...
2,Jaume Collet-Serra (director); Byron Willinger...
3,"Babak Najafi (director); John S. Newman, Chris..."
4,Brett Donowho (director); Nicolas Aaron Mezzan...
...,...
245,"Peter Segal (director); Justin Zackham, Elaine..."
246,Etan Cohen (director/screenplay); Will Ferrell...
247,Adam McKay (director/screenplay); Christian Ba...
248,Mimi Leder (director); Daniel Stiepleman (scre...


In [12]:
data['Title'] = data['Title'].str.lower()

In [13]:
data['Title'][3]

'proud mary'

In [14]:
data = data.rename(columns = {'Title' : 'movie_title'})

In [16]:
!pip install tmdbv3api

In [17]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'c353fd73554c1d566ea8a57b71508242'

In [18]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " "
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.nan

In [20]:
data['genres'] = data['movie_title'].map(lambda x: get_genre(str(x)))

In [21]:
data2 = data[['movie_title', 'Cast and crew', 'genres']]

In [22]:
data2

,movie_title,Cast and crew,genres
0,insidious: the last key,Adam Robitel (director); Leigh Whannell (scree...,Horror Thriller
1,the strange ones,Christopher Radcliff (director/screenplay); La...,Thriller Drama
2,the commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery
3,proud mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime
4,acts of violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Drama Crime Thriller
...,...,...,...
245,second act,"Peter Segal (director); Justin Zackham, Elaine...",Romance Comedy
246,holmes & watson,Etan Cohen (director/screenplay); Will Ferrell...,Drama Mystery Thriller Crime
247,vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure
248,on the basis of sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History


In [47]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (director);" in x:
        return x.split("(director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (directors);" in x:
        return x.split("(directors)")[0]
    elif " (director/screenplay);" in x:
        return x.split("(director/screenplay)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [48]:
data2['director_name'] = data2['Cast and crew'].map(lambda x : get_director(x))

In [49]:
data2

,movie_title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,insidious: the last key,Adam Robitel (director); Leigh Whannell (scree...,Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,the strange ones,Christopher Radcliff (director/screenplay); La...,Thriller Drama,Christopher Radcliff (director/screenplay); La...,Lauren Wolkstein (director); Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,the commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
3,proud mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi,Taraji P. Henson,Jahi Di'Allo Winston,Billy Brown
4,acts of violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Drama Crime Thriller,Brett Donowho,Bruce Willis,Cole Hauser,Shawn Ashmore
...,...,...,...,...,...,...,...
245,second act,"Peter Segal (director); Justin Zackham, Elaine...",Romance Comedy,Peter Segal,Jennifer Lopez,Leah Remini,Vanessa Hudgens
246,holmes & watson,Etan Cohen (director/screenplay); Will Ferrell...,Drama Mystery Thriller Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
247,vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
248,on the basis of sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux


In [50]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [51]:
data2['actor_1_name'] = data2['Cast and crew'].map(lambda x : get_actor1(x))

In [52]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [53]:
data2['actor_2_name'] = data2['Cast and crew'].map(lambda x : get_actor2(x))

In [54]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [55]:
data2['actor_3_name'] = data2['Cast and crew'].map(lambda x : get_actor3(x))

In [56]:
data2

,movie_title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,insidious: the last key,Adam Robitel (director); Leigh Whannell (scree...,Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,the strange ones,Christopher Radcliff (director/screenplay); La...,Thriller Drama,Christopher Radcliff (director/screenplay); La...,Lauren Wolkstein (director); Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,the commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
3,proud mary,"Babak Najafi (director); John S. Newman, Chris...",Thriller Action Crime,Babak Najafi,Taraji P. Henson,Jahi Di'Allo Winston,Billy Brown
4,acts of violence,Brett Donowho (director); Nicolas Aaron Mezzan...,Action Drama Crime Thriller,Brett Donowho,Bruce Willis,Cole Hauser,Shawn Ashmore
...,...,...,...,...,...,...,...
245,second act,"Peter Segal (director); Justin Zackham, Elaine...",Romance Comedy,Peter Segal,Jennifer Lopez,Leah Remini,Vanessa Hudgens
246,holmes & watson,Etan Cohen (director/screenplay); Will Ferrell...,Drama Mystery Thriller Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
247,vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
248,on the basis of sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux


In [57]:
data3 = data2.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [58]:
data3['actor_2_name'] = data3['actor_2_name'].replace(np.nan, 'unknown')
data3['actor_3_name'] = data3['actor_3_name'].replace(np.nan, 'unknown')

In [59]:
data3['comb'] = data3['actor_1_name'] + ' ' + data3['actor_2_name'] + ' '+ data3['actor_3_name'] + ' '+ data3['director_name'] +' ' + data3['genres']

In [60]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [62]:
df = pd.concat([df1, df2, df3, df4], ignore_index = True)

In [63]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,J A N U A R Y,4,Rust Creek,IFC Films / Lunacy Productions,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,J A N U A R Y,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,J A N U A R Y,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,J A N U A R Y,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
245,D E C E M B E R,25,Spies in Disguise,20th Century Fox Animation / Blue Sky Studios ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[133]
246,D E C E M B E R,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[226]
247,D E C E M B E R,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[227]
248,D E C E M B E R,25,Just Mercy,Warner Bros. Pictures / Participant,"Destin Daniel Cretton (director/screenplay), A...",[228]


In [64]:
df['Title'] = df['Title'].str.lower()
df = df.rename(columns = {'Title' : 'movie_title'})

In [65]:
df['genres'] = df['movie_title'].map(lambda x: get_genre(str(x)))

In [67]:
df_2019 = df[['movie_title','Cast and crew','genres']]

In [68]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [69]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-69-203239715>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))


In [70]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [71]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))

<ipython-input-71-1562076556>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))


In [74]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [75]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))

<ipython-input-75-3736213850>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))


In [76]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [77]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [78]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [79]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

In [80]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']

In [81]:
finaldata1 = pd.concat([data3, new_df19], ignore_index = True)

In [82]:
old_df = pd.read_csv('/content/drive/MyDrive/dataset/new_data.csv')

In [83]:
finalData = pd.concat([old_df, finaldata1], ignore_index = True)

In [84]:
finalData

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5425,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5426,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Romance Drama Family,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5427,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War History Thriller Drama,1917,George MacKay Dean-Charles Chapman Mark Strong...
5428,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [86]:
finalData.isna().sum()

,0
director_name,0
actor_1_name,0
actor_2_name,0
actor_3_name,0
genres,3
movie_title,0
comb,3


In [87]:
finalData = finalData.dropna(how='any')

In [88]:
finalData.isna().sum()

,0
director_name,0
actor_1_name,0
actor_2_name,0
actor_3_name,0
genres,0
movie_title,0
comb,0


In [89]:
finalData.to_csv('/content/drive/MyDrive/dataset/final_data.csv',index=False)